# Machine Learning project


In [ ]:
pip install -r requirements.txt

### imports

In [ ]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

from sklearn.model_selection import StratifiedKFold

from score import result

from pathlib import Path


from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Score_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch
import numpy as np
import matplotlib.pyplot as plt

import misvm
import time
from loader import parse_c45, bag_set

## MI-net

In [ ]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    result = []
    for ibatch, batch in enumerate(test_set):
        predicted = model.predict_on_batch({'input':batch[0].astype(np.float32)})[0]

        act = list(batch[1].astype(np.float32))
        if predicted > 0.5:
            result += [1]
        else:
            result += [-1]
    return result

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set..astype(np.float32)
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch][0] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net(X_train, X_test,y_train, y_test,fold):
    """Train and evaluate on MI-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_set = []
    test_set = []
    
    batch_num = len(X_train)
    for ibag, bag in enumerate(X_train):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        train_set.append((batch_data, batch_label))
        
    batch_num = len(X_test)
    for ibag, bag in enumerate(X_test):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        test_set.append((batch_data, batch_label))
        
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # features pooling
    fp = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp')(dropout)

    model = Model(inputs=[data_input], outputs=[fp])
    sgd = SGD(learning_rate=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    start = time.time()
    predicted = []
    for epoch in range(max_epoch):
        print("epoch",epoch)
        train_loss, train_acc = train_eval(model, train_set)
        predicted = test_eval(model, test_set)
    time_ep = time.time() - start
    result("MI-Net", predicted,y_test, time_ep, fold, "musk1")


In [ ]:
for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast','musk1']:
    example_set = parse_c45(dataset)

    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        normed = ((ex - data_mean) / data_std)
        if dataset == "musk1":
            normed[2:-1]
        return normed[1:-1]


    # Group examples into bags
    bagset = bag_set(example_set)

    # Convert bags to NumPy arrays
    bags = [np.array(b.to_float(normalizer)) for b in bagset]
    labels = np.array([b.label for b in bagset], dtype=float)
    # Convert 0/1 labels to -1/1 labels
    labels = 2 * labels - 1

    # perform five times 10-fold cross-validation experiments
    run = 5
    n_folds = 5

    labels = np.array(labels,dtype=int)

    bags = np.array(bags,dtype=object)

    for irun in range(run):
        fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
        splittt = 1
        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            print('run=', irun, '  fold=', splittt)
            MI_Net(X_train, X_test,y_train, y_test,splittt)          
            splittt += 1


# MI-SVM and mi-SVM

In [ ]:
# Load list of C4.5 Examples
#
for i in range(4):
    for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast', 'musk1']:
        example_set = parse_c45(dataset)
        
        # Get stats to normalize data
        raw_data = np.array(example_set.to_float())
        data_mean = np.average(raw_data, axis=0)
        data_std  = np.std(raw_data, axis=0)
        data_std[np.nonzero(data_std == 0.0)] = 1.0
        def normalizer(ex):
            ex = np.array(ex)
            normed = ((ex - data_mean) / data_std)
            # The ...[:, 2:-1] removes first two columns and last column,
            # which are the bag/instance ids and class label, as part of the
            # normalization process
            if dataset == "musk1":
                normed[2:-1]
            return normed[1:-1]

        # Group examples into bags
        bagset = bag_set(example_set)

        # Convert bags to NumPy arrays
        bags = [np.array(b.to_float(normalizer)) for b in bagset]
        labels = np.array([b.label for b in bagset], dtype=float)
        # Convert 0/1 labels to -1/1 labels
        labels = 2 * labels - 1

        # Construct classifiers
        classifiers = {}

        # sMIL    : sparse MIL (Bunescu & Mooney, 2007)
        # stMIL   : sparse, transductive  MIL (Bunescu & Mooney, 2007)
        classifiers['MISVM'] = misvm.MISVM()
        classifiers['miSVM'] = misvm.miSVM()
        #classifiers['MissSVM'] = misvm.MissSVM(kernel='linear', C=1.0, max_iters=20)
        classifiers['sbMIL'] = misvm.sbMIL(kernel='linear', eta=0.1, C=1e2)
        classifiers['SIL'] = misvm.SIL(kernel='linear', C=1.0)
        classifiers['STK'] = misvm.STK(kernel='linear', C=1.0)
        classifiers['NSK'] = misvm.NSK(kernel='linear', C=1.0)
        #classifiers['MICA'] = misvm.MICA(kernel='linear')

        # Train/Evaluate classifiers
        
        bags = np.array(bags,dtype=object)
        labels = np.array(labels,dtype=int)
        fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
        for algorithm, classifier in classifiers.items():
            nums = 1
            start = time.time()
            for train_index, test_index in fold.split(bags,labels):
                X_train, X_test = bags[train_index], bags[test_index]
                y_train, y_test = labels[train_index], labels[test_index]

                classifier.fit(X_train, y_train)
                predictions = classifier.predict(X_test)
                time_ep = time.time() - start
                for i, pred in enumerate(predictions):
                    if pred < 0:
                        predictions[i] = -1
                    else:
                        predictions[i] = 1
                print(algorithm)
                result(algorithm, y_test, predictions, time_ep, nums, dataset)
                nums+=1

Non-random start...

Iteration 1...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -8.6643e+01 -8.2546e-01  2e+03  5e+01  1e-13
 1: -4.7506e+00 -6.8635e-01  1e+02  3e+00  1e-13
 2: -4.8969e-01 -6.1713e-01  9e+00  2e-01  1e-14
 3: -2.1973e-01 -5.5853e-01  5e-01  4e-03  1e-15
 4: -2.3135e-01 -3.0320e-01  8e-02  4e-04  6e-16
 5: -2.4534e-01 -2.7046e-01  3e-02  3e-05  8e-16
 6: -2.5041e-01 -2.5696e-01  7e-03  3e-06  2e-15
 7: -2.5264e-01 -2.5425e-01  2e-03  3e-17  1e-14
 8: -2.5322e-01 -2.5374e-01  5e-04  2e-17  5e-14
 9: -2.5342e-01 -2.5360e-01  2e-04  6e-17  1e-13
10: -2.5352e-01 -2.5354e-01  2e-05  2e-17  9e-14
11: -2.5353e-01 -2.5353e-01  2e-06  1e-17  3e-13
12: -2.5353e-01 -2.5353e-01  8e-08  3e-17  2e-12
Optimal solution found.
Recomputing classes...
Selector differences: 623
Updating QP...

Iteration 2...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -7.0753e+01 -7.3121e-01  3e+03  5e+01  1e-13
 1: -2.9048e+00 -6.7979e-01  9e+01  2e+00  1e-

 7: -2.3343e-01 -2.3344e-01  2e-05  2e-12  6e-02
 8: -2.3349e-01 -2.3351e-01  2e-05  2e-12  5e-02
 9: -2.3357e-01 -2.3360e-01  3e-05  1e-12  4e-02
10: -2.3357e-01 -2.3360e-01  3e-05  1e-12  4e-02
11: -2.3360e-01 -2.3364e-01  3e-05  1e-12  3e-02
12: -2.3366e-01 -2.3369e-01  4e-05  6e-13  2e-02
13: -2.3368e-01 -2.3370e-01  3e-05  4e-13  1e-02
14: -2.3368e-01 -2.3371e-01  3e-05  3e-13  9e-03
15: -2.3368e-01 -2.3370e-01  2e-05  5e-15  2e-04
16: -2.3369e-01 -2.3370e-01  8e-07  2e-16  5e-06
17: -2.3370e-01 -2.3370e-01  1e-08  1e-17  6e-08
Optimal solution found.
Recomputing classes...
Selector differences: 4
Updating QP...

Iteration 6...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -2.3357e-01 -2.3357e-01  1e-08  1e-17  2e-02
 1: -2.3358e-01 -2.3358e-01  3e-08  2e-17  2e-02
 2: -2.3358e-01 -2.3358e-01  3e-06  6e-17  2e-02
 3: -2.3359e-01 -2.3360e-01  3e-06  1e-16  1e-02
 4: -2.3359e-01 -2.3360e-01  3e-06  1e-16  1e-02
 5: -2.3360e-01 -2.3360e-01  3e-06  1e-16  1e-02
 

 3: -1.9718e-01 -4.3108e-01  3e-01  8e-04  9e-16
 4: -2.0544e-01 -2.9185e-01  9e-02  3e-04  6e-16
 5: -2.1399e-01 -2.5127e-01  4e-02  2e-05  6e-16
 6: -2.2124e-01 -2.3237e-01  1e-02  4e-06  8e-16
 7: -2.2487e-01 -2.2927e-01  4e-03  6e-07  2e-15
 8: -2.2618e-01 -2.2802e-01  2e-03  1e-07  2e-15
 9: -2.2711e-01 -2.2736e-01  3e-04  9e-09  3e-15
10: -2.2726e-01 -2.2728e-01  2e-05  7e-10  1e-14
11: -2.2728e-01 -2.2728e-01  1e-06  3e-11  4e-14
12: -2.2728e-01 -2.2728e-01  2e-08  5e-13  4e-13
Optimal solution found.
Recomputing classes...
Selector differences: 6
Updating QP...

Iteration 5...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -6.1907e+01 -7.3514e-01  3e+03  5e+01  1e-13
 1: -1.3055e+00 -6.6442e-01  4e+01  9e-01  1e-13
 2: -2.4506e-01 -6.0431e-01  2e+00  4e-02  5e-15
 3: -1.9640e-01 -4.2497e-01  2e-01  6e-04  1e-15
 4: -2.0503e-01 -2.8883e-01  9e-02  2e-04  6e-16
 5: -2.1405e-01 -2.4795e-01  3e-02  1e-05  7e-16
 6: -2.2128e-01 -2.3116e-01  1e-02  2e-06  1e-15
 

MISVM
Non-random start...

Iteration 1...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -3.8595e+02 -2.7706e+00  8e+03  9e+01  1e-12
 1: -1.3922e+01 -2.6937e+00  3e+02  3e+00  8e-13
 2: -1.5893e+00 -2.4929e+00  2e+01  2e-01  6e-14
 3: -6.6533e-01 -2.1524e+00  2e+00  3e-03  4e-15
 4: -7.4193e-01 -1.0584e+00  3e-01  6e-04  2e-15
 5: -8.2432e-01 -8.9978e-01  8e-02  7e-05  3e-15
 6: -8.4365e-01 -8.6793e-01  2e-02  2e-05  2e-15
 7: -8.4938e-01 -8.5916e-01  1e-02  4e-06  3e-15
 8: -8.5215e-01 -8.5527e-01  3e-03  2e-08  3e-15
 9: -8.5349e-01 -8.5375e-01  3e-04  7e-10  3e-15
10: -8.5361e-01 -8.5362e-01  1e-05  3e-11  3e-15
11: -8.5362e-01 -8.5362e-01  2e-07  4e-13  3e-15
Optimal solution found.
Recomputing classes...
Class Changes: 126

Iteration 2...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -2.9489e+02 -2.5618e+00  8e+03  9e+01  6e-13
 1: -1.2399e+01 -2.4905e+00  3e+02  4e+00  6e-13
 2: -1.3813e+00 -2.3763e+00  2e+01  2e-01  5e-14
 3: -5.6264e-

 6: -1.5796e-01 -2.2919e-01  7e-02  1e-04  5e-16
 7: -1.6182e-01 -2.0286e-01  4e-02  6e-05  4e-16
 8: -1.6506e-01 -1.8573e-01  2e-02  2e-05  4e-16
 9: -1.6618e-01 -1.8119e-01  2e-02  1e-05  4e-16
10: -1.6787e-01 -1.7516e-01  7e-03  5e-06  4e-16
11: -1.6863e-01 -1.7288e-01  4e-03  2e-06  4e-16
12: -1.6913e-01 -1.7167e-01  3e-03  1e-06  4e-16
13: -1.6951e-01 -1.7081e-01  1e-03  4e-07  4e-16
14: -1.6984e-01 -1.7017e-01  3e-04  7e-08  5e-16
15: -1.6994e-01 -1.7002e-01  8e-05  1e-08  4e-16
16: -1.6997e-01 -1.6997e-01  3e-06  3e-11  5e-16
17: -1.6997e-01 -1.6997e-01  4e-08  5e-13  5e-16
Optimal solution found.
Recomputing classes...
Class Changes: 2

Iteration 11...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -9.3612e+01 -2.1662e+00  7e+03  9e+01  2e-13
 1: -4.5536e+00 -2.0953e+00  3e+02  4e+00  2e-13
 2: -3.9353e-01 -2.0078e+00  2e+01  2e-01  1e-14
 3: -1.7384e-01 -1.5530e+00  2e+00  1e-02  1e-15
 4: -1.4870e-01 -3.7953e-01  2e-01  6e-04  1e-15
 5: -1.5305e-01 -2.804

...

 5: -1.8534e-01 -3.0220e-01  1e-01  2e-04  6e-16
 6: -1.9138e-01 -2.3677e-01  5e-02  4e-05  5e-16
 7: -1.9313e-01 -2.2577e-01  3e-02  3e-05  4e-16
 8: -1.9511e-01 -2.1454e-01  2e-02  1e-05  4e-16
 9: -1.9665e-01 -2.0773e-01  1e-02  6e-06  4e-16
10: -1.9792e-01 -2.0303e-01  5e-03  2e-06  4e-16
11: -1.9840e-01 -2.0168e-01  3e-03  1e-06  4e-16
12: -1.9899e-01 -2.0023e-01  1e-03  3e-07  5e-16
13: -1.9924e-01 -1.9968e-01  4e-04  5e-08  5e-16
14: -1.9938e-01 -1.9947e-01  1e-04  8e-09  5e-16
15: -1.9941e-01 -1.9942e-01  1e-05  4e-10  5e-16
16: -1.9942e-01 -1.9942e-01  4e-07  1e-11  6e-16
17: -1.9942e-01 -1.9942e-01  5e-09  2e-13  6e-16
Optimal solution found.
Recomputing classes...
Class Changes: 25

Iteration 6...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -1.0990e+02 -2.1728e+00  8e+03  9e+01  2e-13
 1: -2.4020e+00 -2.1481e+00  2e+02  2e+00  2e-13
 2: -3.0163e-01 -2.0447e+00  9e+00  8e-02  9e-15
 3: -1.7484e-01 -1.1477e+00  1e+00  3e-03  1e-15
 4: -1.7223e-01 -4.572

12: -2.8389e-03 -3.0088e-03  2e-04  2e-16  4e-16
13: -2.8960e-03 -2.9085e-03  1e-05  2e-16  4e-16
14: -2.9018e-03 -2.9019e-03  1e-07  2e-16  5e-16
15: -2.9018e-03 -2.9018e-03  1e-09  2e-16  5e-16
Optimal solution found.
Computing initial instance labels for sbMIL...
Retraining with top 10% as positive...
     pcost       dcost       gap    pres   dres
 0: -1.0172e+02 -1.9010e+02  7e+03  3e+01  2e-13
 1: -1.7503e+01 -1.7506e+02  5e+02  1e+00  2e-13
 2: -1.1514e+01 -9.4275e+01  1e+02  2e-01  4e-14
 3: -9.5247e+00 -5.8999e+01  7e+01  1e-01  3e-14
 4: -8.2926e+00 -2.4642e+01  2e+01  3e-02  2e-14
 5: -8.2074e+00 -1.4683e+01  8e+00  9e-03  2e-14
 6: -8.6304e+00 -1.0701e+01  2e+00  2e-03  3e-14
 7: -8.7979e+00 -1.0083e+01  1e+00  9e-04  3e-14
 8: -8.9625e+00 -9.5302e+00  6e-01  2e-04  3e-14
 9: -9.0994e+00 -9.2554e+00  2e-01  3e-05  3e-14
10: -9.1427e+00 -9.1822e+00  4e-02  2e-16  3e-14
11: -9.1595e+00 -9.1632e+00  4e-03  2e-15  3e-14
12: -9.1611e+00 -9.1613e+00  2e-04  1e-15  3e-14
13: -9.16

12: -8.4645e-01 -8.4645e-01  6e-08  1e-11  3e-15
Optimal solution found.
SIL
     pcost       dcost       gap    pres   dres
 0: -3.4039e+02 -2.7811e+00  7e+03  9e+01  7e-13
 1: -1.4212e+01 -2.5844e+00  3e+02  3e+00  7e-13
 2: -1.6547e+00 -2.4037e+00  2e+01  2e-01  6e-14
 3: -6.6623e-01 -2.1224e+00  2e+00  1e-02  5e-15
 4: -6.9740e-01 -1.0097e+00  3e-01  1e-03  2e-15
 5: -7.8029e-01 -8.5463e-01  8e-02  3e-04  2e-15
 6: -8.0162e-01 -8.2809e-01  3e-02  6e-05  2e-15
 7: -8.0977e-01 -8.1791e-01  8e-03  1e-05  2e-15
 8: -8.1274e-01 -8.1428e-01  2e-03  2e-06  3e-15
 9: -8.1343e-01 -8.1348e-01  6e-05  1e-16  3e-15
10: -8.1345e-01 -8.1345e-01  2e-06  7e-17  3e-15
11: -8.1345e-01 -8.1345e-01  3e-08  1e-16  3e-15
Optimal solution found.
SIL
     pcost       dcost       gap    pres   dres
 0: -3.8183e+02 -2.6729e+00  8e+03  9e+01  9e-13
 1: -8.2009e+00 -2.6302e+00  2e+02  2e+00  9e-13
 2: -1.3012e+00 -2.4673e+00  1e+01  2e-01  8e-14
 3: -6.4728e-01 -2.0305e+00  1e+00  4e-04  4e-15
 4: -7.4955e-01

C:\Users\seven\Documents\github\MachineLearning\score.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(10, 10))


     pcost       dcost       gap    pres   dres
 0: -1.4966e+01 -4.2436e+00  1e+03  3e+01  1e-13
 1: -3.0236e+00 -2.6555e+00  6e+01  2e+00  1e-13
 2: -6.7292e-01 -2.1017e+00  5e+00  1e-01  2e-14
 3: -5.0012e-01 -1.2115e+00  8e-01  6e-03  4e-15
 4: -5.3033e-01 -6.3959e-01  1e-01  8e-04  4e-15
 5: -5.5216e-01 -5.7404e-01  2e-02  1e-04  4e-15
 6: -5.5801e-01 -5.6208e-01  4e-03  2e-05  4e-15
 7: -5.5930e-01 -5.5972e-01  4e-04  1e-06  4e-15
 8: -5.5945e-01 -5.5947e-01  2e-05  4e-08  4e-15
 9: -5.5946e-01 -5.5946e-01  5e-07  9e-10  5e-15
Optimal solution found.
STK
     pcost       dcost       gap    pres   dres
 0: -1.8145e+01 -4.3193e+00  1e+03  3e+01  1e-13
 1: -3.2475e+00 -2.7915e+00  7e+01  2e+00  1e-13
 2: -7.4909e-01 -2.1940e+00  5e+00  1e-01  2e-14
 3: -5.4413e-01 -1.3196e+00  9e-01  7e-03  5e-15
 4: -5.7849e-01 -7.0602e-01  1e-01  1e-03  4e-15
 5: -6.0264e-01 -6.3981e-01  4e-02  3e-04  4e-15
 6: -6.1259e-01 -6.1766e-01  5e-03  8e-17  5e-15
 7: -6.1421e-01 -6.1477e-01  6e-04  1e-16  

# mi-Net

In [ ]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    result = []
    for ibatch, batch in enumerate(test_set):
        predicted = model.predict_on_batch({'input':batch[0].astype(np.float32)})[0]
        act = list(batch[1].astype(np.float32))
        if predicted > 0.5:
            result += [1]
        else:
            result += [-1]
    return result


def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def mi_Net(X_train, X_test,y_train, y_test,fold):
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=25
    """Train and evaluate on mi-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of mi-Net.
    """
    train_set = []
    test_set = []
    
    batch_num = len(X_train)
    for ibag, bag in enumerate(X_train):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        train_set.append((batch_data, batch_label))
        
    batch_num = len(X_test)
    for ibag, bag in enumerate(X_test):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        test_set.append((batch_data, batch_label))
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # score pooling
    sp = Score_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='sp')(dropout)

    model = Model(inputs=[data_input], outputs=[sp])
    sgd = SGD(learning_rate=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    start = time.time()
    predicted = []
    for epoch in range(max_epoch):
        print("epoch",epoch)
        train_loss, train_acc = train_eval(model, train_set)
        predicted = test_eval(model, test_set)
    time_ep = time.time() - start
    result("mi-Net", predicted,y_test, time_ep, fold, "musk1")


In [ ]:
for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast','musk1']:
    example_set = parse_c45(dataset)

    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        normed = ((ex - data_mean) / data_std)
        if dataset == "musk1":
            normed[2:-1]
        return normed[1:-1]


    # Group examples into bags
    bagset = bag_set(example_set)

    # Convert bags to NumPy arrays
    bags = [np.array(b.to_float(normalizer)) for b in bagset]
    labels = np.array([b.label for b in bagset], dtype=float)
    # Convert 0/1 labels to -1/1 labels
    labels = 2 * labels - 1

    # perform five times 10-fold cross-validation experiments
    run = 5
    n_folds = 5

    labels = np.array(labels,dtype=int)

    bags = np.array(bags,dtype=object)

    for irun in range(run):
        fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
        splittt = 1
        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            print('run=', irun, '  fold=', splittt)
            mi_Net(X_train, X_test,y_train, y_test,splittt)          
            splittt += 1

In [ ]:
import pandas as pd
data = pd.read_csv("compare/score.csv")
set(data[data["pc"]=="DESKTOP-KB1MVGV"].algo)